In [2]:
# import Python libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import Keys

import time, pandas as pd
from bs4 import BeautifulSoup

from datetime import datetime, date 
from datetime import timedelta 

In [3]:
# retrieve youtube channels
with open('inputs.txt') as file:
    url = file.read()

In [4]:
# open chrome browser and load the page
driver = webdriver.Chrome()
driver.get(url)

In [5]:
# find the html element
html = driver.find_element(By.TAG_NAME, "html")

In [6]:
# Find the total number of videos
video_count = driver.find_element(By.XPATH, '//*[@id="videos-count"]').text.split()[0]

# Calculate the loop count
video_loop_length = round(int(video_count)/(4*7)) + 1

In [7]:
# Load all the videos 
for _ in range(video_loop_length):
    html.send_keys(Keys.END)
    time.sleep(3)

In [8]:
# Get the page source
page = driver.page_source

# Retrieve the html elements
soup = BeautifulSoup(page, 'html')

# get the html element containing the videos
videos = soup.find_all('div', {'id': 'dismissible'})

In [10]:
# creating a list of video info records
master_list = []
for item in get_video(videos):
  master_list.append(item)

229

In [9]:
# function to retrieve the data 
def get_video(videos):
    for video in videos:
        data_dict = {}
        data_dict['title'] = video.find('a', {'id': 'video-title-link'}).text
        data_dict['video_url'] = 'https://www.youtube.com' + video.find('a', {'id': 'video-title-link'})['href']
        meta = video.find('div', {'id': 'metadata-line'}).find_all('span')
        data_dict['views'] = meta[0].text
        data_dict['video_age'] = meta[1].text
        
        yield data_dict

In [28]:
# declare and assign a pandas DataFrame variable
df = pd.DataFrame(master_list)

In [14]:
# create a function to clean up the data
def convert_views(df):
    views = ''
    if 'K' in df['views']:
        views = float(df['views'].split('K')[0]) * 1000
    elif 'M' in df['views']:
        views = float(df['views'].split('M')[0]) * 1000000
    return views

In [29]:
# apply the function to dataframe
# first define a column in dataframe
df['cleaned_views'] = df.apply(convert_views, axis=1)
df['cleaned_views'] = df['cleaned_views'].astype(int)

In [30]:
# get date published
def get_date_published(df):
    date_pub = ''
    num = int(df['video_age'].split()[0])
    period = df['video_age'].split()[1]
    
    if period == 'days':
        date_pub = pd.to_datetime(date.today()) - pd.DateOffset(days=num)
    elif period in ['weeks', 'week']:
        date_pub = pd.to_datetime(date.today()) - pd.DateOffset(weeks=num)
    elif period in ['month', 'months']:
        date_pub = pd.to_datetime(date.today()) - pd.DateOffset(months=num)
    elif period in ['years', 'year']:
        date_pub = pd.to_datetime(date.today()) - pd.DateOffset(years=num)
    return date_pub.strftime("%d %b, %Y")

In [31]:
# apply the get_date_published function
# first define a column in dataframe
df['date_published'] = df.apply(get_date_published, axis=1)

In [37]:
# Select the and re-arrange the required columns
new_df = df[['title', 'video_url', 'views', 'date_published']]

# Save report as excel file
new_df.to_excel('sample_output.xlsx', index=False)

# Save report as csv file
new_df.to_excel('sample_output.csv', index=False)